# ReAct

In [6]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType, load_tools
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

In [44]:
# TODO: Was sind die eigentlichen Prompts
# TODO: beispiel mit suche nach HackyHour

llm = ChatOpenAI(model="gpt-4-0613")

tools = load_tools(["serpapi", "llm-math"], llm=llm)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
    memory=memory,
    verbose=True
)

In [54]:
agent_chain.agent

ConversationalChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], input_types={'chat_history': typing.List[typing.Union[langchain.schema.messages.AIMessage, langchain.schema.messages.HumanMessage, langchain.schema.messages.ChatMessage, langchain.schema.messages.SystemMessage, langchain.schema.messages.FunctionMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain.schema.messages.AIMessage, langchain.schema.messages.HumanMessage, langchain.schema.messages.ChatMessage, langchain.schema.messages.SystemMessage, langchain.schema.messages.FunctionMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is a

In [32]:
agent_chain.agent.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of top

In [33]:
for i in agent_chain.agent.llm_chain.prompt.messages:
    try:
        print(i.prompt.template)
        print("############################")
    except:
        pass

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

In [60]:
agent_chain.agent.create_prompt(human_message="Who is leonardo di caprios current girlfriend and what is here age multiplied by 3?", tools=tools, input_variables=['chat_history', 'agent_scratchpad']).messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of top

In [47]:
agent_chain.invoke("Who is leonardo di caprios current girlfriend and what is here age multiplied by 3?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "Leonardo DiCaprio's current girlfriend"
}
```
Observation: Leonardo DiCaprio and new girlfriend Vittoria Ceretti hit another Paris party.
Thought:```json
{
    "action": "Search",
    "action_input": "Vittoria Ceretti's age"
}
```
Observation: 25 years
Thought:```json
{
    "action": "Calculator",
    "action_input": "25 * 3"
}
```
Observation: Answer: 75
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Leonardo DiCaprio's current girlfriend is Vittoria Ceretti, and her age multiplied by 3 is 75."
}
```

> Finished chain.


{'input': 'Who is leonardo di caprios current girlfriend and what is here age multiplied by 3?',
 'chat_history': [HumanMessage(content='Who is leonardo di caprios current girlfriend and what is here age multiplied by 3?'),
  AIMessage(content="Leonardo DiCaprio's current girlfriend is Vittoria Ceretti, and her age multiplied by 3 is 75.")],
 'output': "Leonardo DiCaprio's current girlfriend is Vittoria Ceretti, and her age multiplied by 3 is 75."}

In [127]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613")
# search = SerpAPIWrapper()
# llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
# 
# tools = [
#     Tool(
#         name="Search",
#         func=search.run,
#         description="useful for when you need to answer questions about current events. You should ask targeted questions",
#     ),
#     Tool(
#         name="Calculator",
#         func=llm_math_chain.run,
#         description="useful for when you need to answer questions about math",
#     ),
# ]

tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    # agent_kwargs=agent_kwargs,
    memory=memory,
)